# Preprocessing Claim Data

The goal is to transform Claims data from **claim-based** to **member-based**. Two steps are involved:
- Convert categorical variables (such as PrimaryConditionGroup, CharlsonIndex, etc.) to numerical or binary types
- Aggregate these variable on (year, member) basis

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
from sklearn.preprocessing import MultiLabelBinarizer
warnings.filterwarnings('ignore')

In [25]:
df_claims = pd.read_csv('../data/Claims.csv')

In [26]:
df_claims.dtypes

MemberID                   int64
ProviderID               float64
Vendor                   float64
PCP                      float64
Year                      object
Specialty                 object
PlaceSvc                  object
PayDelay                  object
LengthOfStay              object
DSFS                      object
PrimaryConditionGroup     object
CharlsonIndex             object
ProcedureGroup            object
SupLOS                     int64
dtype: object

In [27]:
df_claims.head()

,MemberID,ProviderID,Vendor,PCP,Year,Specialty,PlaceSvc,PayDelay,LengthOfStay,DSFS,PrimaryConditionGroup,CharlsonIndex,ProcedureGroup,SupLOS
0,42286978,8013252.0,172193.0,37796.0,Y1,Surgery,Office,28,NaN,8- 9 months,NEUMENT,0,MED,0
1,97903248,3316066.0,726296.0,5300.0,Y3,Internal,Office,50,NaN,7- 8 months,NEUMENT,1-2,EM,0
2,2759427,2997752.0,140343.0,91972.0,Y3,Internal,Office,14,NaN,0- 1 month,METAB3,0,EM,0
3,73570559,7053364.0,240043.0,70119.0,Y3,Laboratory,Independent Lab,24,NaN,5- 6 months,METAB3,1-2,SCS,0
4,11837054,7557061.0,496247.0,68968.0,Y2,Surgery,Outpatient Hospital,27,NaN,4- 5 months,FXDISLC,1-2,EM,0


In [5]:
def count_nulls(df):
    null_counter = df.isnull().sum(axis=0)
    #null_counter = null_counter[null_counter > 0]
    null_percent = df.isnull().sum(axis=0) / df.shape[0] * 100
    #null_percent = null_percent[null_percent > 0]
    null_df = pd.concat([null_counter,null_percent],axis=1)
    null_df.columns = ['null_count','null_percent']
    display(null_df)
    
count_nulls(df_claims)

,null_count,null_percent
MemberID,0,0.000000
ProviderID,16264,0.609369
Vendor,24856,0.931289
PCP,7492,0.280705
Year,0,0.000000
Specialty,8405,0.314913
PlaceSvc,7632,0.285951
PayDelay,0,0.000000
LengthOfStay,2597392,97.317412
DSFS,52770,1.977152


In [28]:
# Create new variable LOS_TOT_UNKNOWN

def createLTU():
    df_claims['LOS_TOT_UNKNOWN'] = np.where((df_claims['LengthOfStay'].isnull()) & (df_claims['SupLOS']==0), 1, 0)

createLTU()

In [29]:
# Create new variable LOS_TOT_SUPRESSED

def createLTS():
    df_claims['LOS_TOT_SUPRESSED'] = np.where((df_claims['LengthOfStay'].isnull()) & (df_claims['SupLOS']==1), 1, 0)

createLTS()

In [30]:
# Create new variable LOS_TOT_KNOWN

def createLTK():
    df_claims['LOS_TOT_KNOWN'] = np.where(df_claims['LengthOfStay'].isnull(), 0, 1)

createLTK()

In [36]:
# PayDelay
   # Convert the value 162+ of varibale PayDelay into numerical type

def updatePayDelay():
    df_claims.loc[df_claims['PayDelay']=='162+','PayDelay'] = 162
    df_claims['PayDelay'] = df_claims['PayDelay'].astype(int)
    return

updatePayDelay()

In [37]:
# DSFS 
   # Convert from categorical to numerical type
def updateDSFS():
    df_claims.loc[df_claims['DSFS']=='0- 1 month','DSFS'] = 15
    df_claims.loc[df_claims['DSFS']=='1- 2 months','DSFS'] = 45
    df_claims.loc[df_claims['DSFS']=='2- 3 months','DSFS'] = 75
    df_claims.loc[df_claims['DSFS']=='3- 4 months','DSFS'] = 105
    df_claims.loc[df_claims['DSFS']=='4- 5 months','DSFS'] = 135
    df_claims.loc[df_claims['DSFS']=='5- 6 months','DSFS'] = 165
    df_claims.loc[df_claims['DSFS']=='6- 7 months','DSFS'] = 195
    df_claims.loc[df_claims['DSFS']=='7- 8 months','DSFS'] = 225
    df_claims.loc[df_claims['DSFS']=='8- 9 months','DSFS'] = 255
    df_claims.loc[df_claims['DSFS']=='9-10 months','DSFS'] = 285
    df_claims.loc[df_claims['DSFS']=='10-11 months','DSFS'] = 315
    df_claims.loc[df_claims['DSFS']=='11-12 months','DSFS'] = 345
    df_claims['DSFS'].fillna(0, inplace=True)
    df_claims['DSFS'] = df_claims['DSFS'].astype(int)
    return

updateDSFS()

In [38]:
# CharlsonIndex
   # Convert from categorical to numerical type

def updateCharlsonIndex():
    df_claims.loc[df_claims['CharlsonIndex']=='0','CharlsonIndex'] = 0
    df_claims.loc[df_claims['CharlsonIndex']=='1-2','CharlsonIndex'] = 1
    df_claims.loc[df_claims['CharlsonIndex']=='3-4','CharlsonIndex'] = 2
    df_claims.loc[df_claims['CharlsonIndex']=='5+','CharlsonIndex'] = 3

    df_claims['CharlsonIndex'] = df_claims['CharlsonIndex'].astype(int)
    return
    
updateCharlsonIndex()

In [39]:
# LengthOfStay
   # Convert from categorical to numerical

def updateLengthOfStay():
    df_claims.loc[df_claims['LengthOfStay']=='1 day','LengthOfStay'] = 1
    df_claims.loc[df_claims['LengthOfStay']=='2 days','LengthOfStay'] = 2
    df_claims.loc[df_claims['LengthOfStay']=='3 days','LengthOfStay'] = 3
    df_claims.loc[df_claims['LengthOfStay']=='4 days','LengthOfStay'] = 4
    df_claims.loc[df_claims['LengthOfStay']=='5 days','LengthOfStay'] = 5
    df_claims.loc[df_claims['LengthOfStay']=='6 days','LengthOfStay'] = 6
    df_claims.loc[df_claims['LengthOfStay']=='1- 2 weeks','LengthOfStay'] = 11
    df_claims.loc[df_claims['LengthOfStay']=='2- 4 weeks','LengthOfStay'] = 21
    df_claims.loc[df_claims['LengthOfStay']=='4- 8 weeks','LengthOfStay'] = 42
    df_claims.loc[df_claims['LengthOfStay']=='26+ weeks','LengthOfStay'] = 182
    
    df_claims['LengthOfStay'].fillna(0, inplace=True)

    df_claims['LengthOfStay'] = df_claims['LengthOfStay'].astype(int)
    return

updateLengthOfStay()

In [40]:
# PrimaryConditionGroup
   # One Hot Encoding

list_pcg = ['AMI', 'APPCHOL', 'ARTHSPIN', 'CANCRA', 'CANCRB', 'CANCRM', 
            'CATAST', 'CHF', 'COPD', 'FLaELEC', 'FXDISLC', 'GIBLEED', 
            'GIOBSENT', 'GYNEC1', 'GYNECA', 'HEART2', 'HEART4', 
            'HEMTOL', 'HIPFX', 'INFEC4', 'LIVERDZ', 'METAB1', 
            'METAB3', 'MISCHRT', 'MISCL1', 'MISCL5', 'MSC2a3', 
            'NEUMENT', 'ODaBNCA', 'PERINTL', 'PERVALV', 'PNCRDZ', 
            'PNEUM', 'PRGNCY', 'RENAL1', 'RENAL2', 'RENAL3', 
            'RESPR4', 'ROAMI', 'SEIZURE', 'SEPSIS', 'SKNAUT', 
            'STROKE', 'TRAUMA', 'UTI']

def updatePCG():
    index = 1
    for pcg in list_pcg:
        df_claims['pcg'+str(index)] = np.where(df_claims['PrimaryConditionGroup']==pcg, 1, 0)
        index+=1
    df_claims['pcg'+str(index)] = np.where(df_claims['PrimaryConditionGroup'].isnull(), 1, 0)

updatePCG()

In [41]:
# Specialty
   # One Hot Encoding

list_specialty = list(df_claims['Specialty'].unique())

def updateSpecialty():
    index = 1
    for sp in list_specialty:
        df_claims['sp'+str(index)] = np.where(df_claims['Specialty']==sp, 1, 0)
        index+=1
    df_claims['sp'+str(index)] = np.where(df_claims['Specialty'].isnull(), 1, 0)

updateSpecialty()

In [98]:
df_claims['Specialty'].unique()

array(['Surgery', 'Internal', 'Laboratory', 'Pediatrics',
       'Rehabilitation', 'Diagnostic Imaging', 'Anesthesiology',
       'Emergency', 'General Practice', 'Other',
       'Obstetrics and Gynecology', 'Pathology', nan], dtype=object)

In [42]:
# ProcedureGroup
   # One Hot Encoding

list_pg = ['ANES', 'EM', 'MED', 'PL', 'RAD', 'SAS', 'SCS', 'SDS', 'SEOA',
           'SGS', 'SIS', 'SMCD', 'SMS', 'SNS', 'SO', 'SRS', 'SUS']

def updatePG():
    index = 1
    for pg in list_pg:
        df_claims['pg'+str(index)] = np.where(df_claims['ProcedureGroup']==pg, 1, 0)
        index+=1
    df_claims['pg'+str(index)] = np.where(df_claims['ProcedureGroup'].isnull(), 1, 0)

updatePG()

In [43]:
# PlaceSvc
   # One Hot Encoding

list_ps = list(df_claims['PlaceSvc'].unique())

def updatePS():
    index = 1
    for ps in list_ps:
        df_claims['ps'+str(index)] = np.where(df_claims['PlaceSvc']==ps, 1, 0)
        index+=1
    df_claims['ps'+str(index)] = np.where(df_claims['PlaceSvc'].isnull(), 1, 0)

updatePS()

In [99]:
df_claims['PlaceSvc'].unique()

array(['Office', 'Independent Lab', 'Outpatient Hospital',
       'Inpatient Hospital', 'Urgent Care', 'Ambulance', 'Home', nan,
       'Other'], dtype=object)

In [55]:
#Aggregation on (Year, Member)
   #Helper functions
def std(x):
    if len(x) == 1:
        return 0
    else:
        return np.std(x)
    
def rangeM(x):
    return np.max(x) - np.min(x)

   #Aggregate
df_claims_summary = df_claims.groupby(['Year','MemberID']).agg({
    'MemberID': 'count',
    'ProviderID': 'nunique',
    'Vendor': 'nunique',
    'PCP': 'nunique',
    'PlaceSvc': 'nunique',
    'Specialty': 'nunique',
    'PrimaryConditionGroup': 'nunique',
    'ProcedureGroup': 'nunique',
    'PayDelay': ['sum','max','min', 'mean', std],
    'LengthOfStay': ['sum','max','min','mean', std],
    'LOS_TOT_UNKNOWN': 'sum',
    'LOS_TOT_SUPRESSED': 'sum',
    'LOS_TOT_KNOWN': 'sum',
    'DSFS': ['sum', 'max', 'min', 'mean', rangeM, std],
    'CharlsonIndex': ['max', 'min', 'mean', rangeM, std],
    'pcg1': 'sum',
    'pcg2': 'sum',
    'pcg3': 'sum',
    'pcg4': 'sum',
    'pcg5': 'sum',
    'pcg6': 'sum',
    'pcg7': 'sum',
    'pcg8': 'sum',
    'pcg9': 'sum',
    'pcg10': 'sum',
    'pcg11': 'sum',
    'pcg12': 'sum',
    'pcg13': 'sum',
    'pcg14': 'sum',
    'pcg15': 'sum',
    'pcg16': 'sum',
    'pcg17': 'sum',
    'pcg18': 'sum',
    'pcg19': 'sum',
    'pcg20': 'sum',
    'pcg21': 'sum',
    'pcg22': 'sum',
    'pcg23': 'sum',
    'pcg24': 'sum',
    'pcg25': 'sum',
    'pcg26': 'sum',
    'pcg27': 'sum',
    'pcg28': 'sum',
    'pcg29': 'sum',
    'pcg30': 'sum',
    'pcg31': 'sum',
    'pcg32': 'sum',
    'pcg33': 'sum',
    'pcg34': 'sum',
    'pcg35': 'sum',
    'pcg36': 'sum',
    'pcg37': 'sum',
    'pcg38': 'sum',
    'pcg39': 'sum',
    'pcg40': 'sum',
    'pcg41': 'sum',
    'pcg42': 'sum',
    'pcg43': 'sum',
    'pcg44': 'sum',
    'pcg45': 'sum',
    'pcg46': 'sum',
    'sp1': 'sum',
    'sp2': 'sum',
    'sp3': 'sum',
    'sp4': 'sum',
    'sp5': 'sum',
    'sp6': 'sum',
    'sp7': 'sum',
    'sp8': 'sum',
    'sp9': 'sum',
    'sp10': 'sum',
    'sp11': 'sum',
    'sp12': 'sum',
    'sp13': 'sum',
    'sp14': 'sum',
    'pg1': 'sum',
    'pg2': 'sum',
    'pg3': 'sum',
    'pg4': 'sum',
    'pg5': 'sum',
    'pg6': 'sum',
    'pg7': 'sum',
    'pg8': 'sum',
    'pg9': 'sum',
    'pg10': 'sum',
    'pg11': 'sum',
    'pg12': 'sum',
    'pg13': 'sum',
    'pg14': 'sum',
    'pg15': 'sum',
    'pg16': 'sum',
    'pg17': 'sum',
    'pg18': 'sum',
    'ps1': 'sum',
    'ps2': 'sum',
    'ps3': 'sum',
    'ps4': 'sum',
    'ps5': 'sum',
    'ps6': 'sum',
    'ps7': 'sum',
    'ps8': 'sum',
    'ps9': 'sum',
    'ps10': 'sum'
})

In [90]:
# Change column names
df_claims_summary.columns = ['no_Claims',
    'no_Providers',
    'no_Vendors',
    'no_PCPs',
    'no_PlaceSvcs',
    'no_Specialties',
    'no_PrimaryConditionGroups',
    'no_ProcedureGroups',
    'PayDelay_sum',
    'PayDelay_max',
    'PayDelay_min', 
    'PayDelay_mean', 
    'PayDelay_std',
    'LengthOfStay_sum', 
    'LengthOfStay_max',
    'LengthOfStay_min', 
    'LengthOfStay_mean', 
    'LengthOfStay_std',                          
    'LOS_TOT_UNKNOWN_count',
    'LOS_TOT_SUPRESSED_count',
    'LOS_TOT_KNOWN_count',                           
    'DSFS_sum', 
    'DSFS_max', 
    'DSFS_min', 
    'DSFS_mean', 
    'DSFS_range', 
    'DSFS_std',
    'CharlsonIndex_max', 
    'CharlsonIndex_min', 
    'CharlsonIndex_mean', 
    'CharlsonIndex_range', 
    'CharlsonIndex_std',
    'pcg1',
    'pcg2',
    'pcg3',
    'pcg4',
    'pcg5',
    'pcg6',
    'pcg7',
    'pcg8',
    'pcg9',
    'pcg10',
    'pcg11',
    'pcg12',
    'pcg13',
    'pcg14',
    'pcg15',
    'pcg16',
    'pcg17',
    'pcg18',
    'pcg19',
    'pcg20',
    'pcg21',
    'pcg22',
    'pcg23',
    'pcg24',
    'pcg25',
    'pcg26',
    'pcg27',
    'pcg28',
    'pcg29',
    'pcg30',
    'pcg31',
    'pcg32',
    'pcg33',
    'pcg34',
    'pcg35',
    'pcg36',
    'pcg37',
    'pcg38',
    'pcg39',
    'pcg40',
    'pcg41',
    'pcg42',
    'pcg43',
    'pcg44',
    'pcg45',
    'pcg46',
    'sp1',
    'sp2',
    'sp3',
    'sp4',
    'sp5',
    'sp6',
    'sp7',
    'sp8',
    'sp9',
    'sp10',
    'sp11',
    'sp12',
    'sp13',
    'sp14',
    'pg1',
    'pg2',
    'pg3',
    'pg4',
    'pg5',
    'pg6',
    'pg7',
    'pg8',
    'pg9',
    'pg10',
    'pg11',
    'pg12',
    'pg13',
    'pg14',
    'pg15',
    'pg16',
    'pg17',
    'pg18',
    'ps1',
    'ps2',
    'ps3',
    'ps4',
    'ps5',
    'ps6',
    'ps7',
    'ps8',
    'ps9',
    'ps10']

# Reset index
df_claims_summary = df_claims_summary.reset_index()

In [92]:
# Export to csv for further processing
df_claims_summary.to_csv("processed-data/claims_summary.csv", index=False)